## 도서 리뷰 장르 분류 AI 미션

### 배경 스토리  
여러분은 온라인 도서 리뷰 플랫폼 **‘북톡(BOOK-Talk)’**의 AI 엔지니어 신입입니다.  
사용자가 작성한 리뷰를 분석하여 다음 세 가지 장르 중 하나로 자동 분류하는 시스템을 만들어야 합니다.  
- **Novel(소설)**  
- **Self-help(자기계발)**  
- **History(역사)**  

이를 위해 FastText 기반 단어 임베딩과, 네 가지 신경망 모델(**Vanilla FC**, **RNN**, **LSTM**, **GRU**)을 직접 구현하고 성능을 비교해 보세요.

### 데이터셋 다운로드: 

받아서 이름 변경해야.

https://www.kaggle.com/datasets/athu1105/book-genre-prediction?utm_source=chatgpt.com


### ✏️ 기초 코드 설명
데이터 로드

book_reviews.csv에서 리뷰(review)와 장르(genre)를 불러옵니다.

전처리

영어는 소문자화, 한글·영어·숫자·공백만 남겨 토큰화합니다.

FastText 임베딩

gensim.downloader로 사전 학습된 FastText 모델(fasttext-wiki-news-subwords-300)을 로드합니다.

어휘 사전 구축

전체 토큰 빈도 상위 9,999개를 선별해 word2idx 매핑을 만들고, 나머지는 OOV(0) 처리합니다.

최대 길이 100으로 패딩·절단해 시퀀스로 변환합니다.

PyTorch 준비

TensorDataset과 DataLoader로 배치 처리 환경을 구성합니다.

임베딩 레이어 초기화

FastText 벡터로 nn.Embedding 가중치를 채우고, 미세조정은 하지 않도록 고정(requires_grad=False)했습니다.

모델 정의

Vanilla FC: 평균 임베딩 후 완전 연결층으로 분류

RNN/LSTM/GRU: 각 시퀀스를 순차 처리하고 마지막 hidden state를 분류기로 연결

학습·평가

Adam 옵티마이저, CrossEntropyLoss로 5 epoch 동안 학습

테스트 세트 정확도를 비교

결과 해석


이 미션을 통해 FastText 단어 임베딩과 다양한 순환 신경망 구조를 직접 구현·비교해 보세요. 반복 학습을 통해 하이퍼파라미터나 모델 구조를 변형해 보는 과제도 추천합니다.


In [7]:
# 1. 환경 설정 및 데이터 로드
import pandas as pd
from sklearn.model_selection import train_test_split

# 'book_reviews.csv'에는 컬럼 'review', 'genre'가 있습니다.
df = pd.read_csv('book_reviews.csv')
df['review'] = df['summary'] 
df.drop(columns=['summary'], inplace=True)
print(df.shape)      # (샘플 수, 2)
print(df.head())


(4657, 4)
   index                      title    genre  \
0      0          Drowned Wednesday  fantasy   
1      1              The Lost Hero  fantasy   
2      2  The Eyes of the Overworld  fantasy   
3      3            Magic's Promise  fantasy   
4      4             Taran Wanderer  fantasy   

                                              review  
0   Drowned Wednesday is the first Trustee among ...  
1   As the book opens, Jason awakens on a school ...  
2   Cugel is easily persuaded by the merchant Fia...  
3   The book opens with Herald-Mage Vanyel return...  
4   Taran and Gurgi have returned to Caer Dallben...  


In [9]:

# 2. 텍스트 전처리 및 토큰화
import re
def preprocess_text(text):
    # TODO: text를 소문자 변환 후
    text = text.lower()
    # TODO: 특수문자 제거 (한글·영어·숫자 및 공백만 남기기)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    # TODO: 띄어쓰기 기준 토큰화하여 리스트로 반환
    tokens = text.split()
    return tokens

df['tokens'] = df['review'].apply(preprocess_text)

In [12]:

# 3. FastText 임베딩 로드
import gensim.downloader as api
fasttext = api.load("fasttext-wiki-news-subwords-300")
print("벡터 차원:", fasttext.vector_size)


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# 4. 단어 사전(vocab) 구축 및 시퀀스 변환
from collections import Counter
MAX_VOCAB_SIZE = 10000
all_tokens = [tok for tokens in df['tokens'] for tok in tokens]
vocab_count = Counter(all_tokens)
# TODO: 가장 빈도 높은 MAX_VOCAB_SIZE-1개의 단어를 뽑아 인덱스 매핑 생성
# idx 0은 OOV용으로 예약

def tokens_to_sequence(tokens, word2idx, maxlen=100):
    # TODO: 각 토큰을 인덱스로 변환, OOV는 0
    # TODO: 길이 > maxlen이면 자르고, < maxlen이면 0으로 패딩
    return seq

df['seq'] = df['tokens'].apply(lambda toks: tokens_to_sequence(toks, word2idx, maxlen=100))


In [ ]:
# 5. 학습/검증 데이터 준비
import numpy as np
X = np.stack(df['seq'].values)              # (N, maxlen)
y = df['genre'].factorize()[0]              # [0,1,2,...]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:

# 6. PyTorch 모델 구현
import torch
import torch.nn as nn

# 6-1) 임베딩 레이어 초기화
vocab_size = len(word2idx) + 1
embed_dim = fasttext.vector_size

embedding_matrix = torch.zeros(vocab_size, embed_dim)
# TODO: word2idx에 해당하는 fasttext 벡터로 embedding_matrix 채우기
# embedding_matrix[i] = fasttext[word] or zeros

embedding_layer = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
# TODO: embedding_layer.weight.data에 embedding_matrix 할당
# TODO: 임베딩 고정(freeze)할지 결정

# 6-2) 분류기 클래스 정의
class VanillaFC(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        # TODO: 평균 임베딩 벡터 입력 → 예측 로직
        pass

class RNNClassifier(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, num_classes):
        super().__init__()
        # TODO: embedding_layer 복사
        # TODO: nn.RNN 정의
        # TODO: 마지막 시점 hidden → fc
    def forward(self, x):
        pass

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, num_classes):
        super().__init__()
        # TODO: nn.LSTM 정의
    def forward(self, x):
        pass

class GRUClassifier(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, num_classes):
        super().__init__()
        # TODO: nn.GRU 정의
    def forward(self, x):
        pass

In [ ]:


# 7. 학습 및 평가 함수
from torch.utils.data import DataLoader, TensorDataset

def train(model, optimizer, criterion, loader, device):
    model.train()
    # TODO: 배치 반복 → loss 계산 → 역전파 → optimizer.step()

def evaluate(model, loader, device):
    model.eval()
    # TODO: 배치 반복 → 예측 → 정확도 계산 → 평균 반환
    return accuracy


In [ ]:

# 8. 모델 학습 및 성능 비교
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(set(y))
hidden_dim = 64

models = {
    'FC':   VanillaFC(input_dim=embed_dim, num_classes=num_classes),
    'RNN':  RNNClassifier(embedding_layer, hidden_dim, num_classes),
    'LSTM': LSTMClassifier(embedding_layer, hidden_dim, num_classes),
    'GRU':  GRUClassifier(embedding_layer, hidden_dim, num_classes),
}

for name, model in models.items():
    model.to(device)
    # TODO: optimizer, criterion 설정
    # TODO: train(model, ...) 호출
    # TODO: val_acc = evaluate(model, ...) 출력
